In [4]:
from pathlib import Path

# Define folders
pdf_folder = Path("failed_pdfs")
txt_folder = Path("txt_final")

# Get list of .txt base names
txt_basenames = {txt.stem for txt in txt_folder.glob("*.txt")}

# Loop over PDFs and delete if corresponding TXT exists
deleted_count = 0
for pdf in pdf_folder.glob("*.pdf"):
    if pdf.stem in txt_basenames:
        print(f"🗑 Deleting {pdf.name} (already processed)")
        pdf.unlink()
        deleted_count += 1

print(f"\n✅ Done. Deleted {deleted_count} file(s).")


✅ Done. Deleted 0 file(s).


In [3]:
# 🛠 Required packages:
# pip install pytesseract pdf2image pillow tqdm

import pytesseract
from pdf2image import convert_from_path
from tqdm import tqdm
from pathlib import Path
import shutil
import gc  # For garbage collection

# 📁 Define folders
input_folder = Path("proc_final")            # Folder with input PDFs
output_folder = Path("txt_final")            # Output folder for TXT files
failed_folder = Path("failed_pdfs")          # Optional: store failed PDFs

output_folder.mkdir(parents=True, exist_ok=True)
failed_folder.mkdir(parents=True, exist_ok=True)

# 🧠 OCR Function
def ocr_pdf_to_txt(pdf_path, txt_path):
    try:
        print(f"🔍 Processing {pdf_path.name}")
        # Convert PDF to list of page images
        pages = convert_from_path(pdf_path)

        if not pages:
            print(f"⚠️ No pages found in {pdf_path.name}. Skipping.")
            return

        with open(txt_path, "w", encoding="utf-8") as out:
            for page_number, page in enumerate(
                tqdm(pages, desc=f"OCR: {pdf_path.name}", leave=False), start=1
            ):
                text = pytesseract.image_to_string(page)
                out.write(f"\n\n--- Page {page_number} ---\n\n")
                out.write(text)
                del page
                gc.collect()

    except Exception as e:
        print(f"❌ Failed to process {pdf_path.name}: {e}")
        try:
            shutil.move(str(pdf_path), failed_folder / pdf_path.name)
            print(f"📁 Moved to failed_pdfs: {pdf_path.name}")
        except Exception as move_error:
            print(f"⚠️ Could not move failed file: {move_error}")

# 🔁 Loop through all PDFs
pdf_files = list(input_folder.glob("*.pdf"))
print(f"📚 Found {len(pdf_files)} PDF(s) in {input_folder}")

for pdf_file in tqdm(pdf_files, desc="📦 Processing PDFs"):
    txt_file = output_folder / f"{pdf_file.stem}.txt"
    ocr_pdf_to_txt(pdf_file, txt_file)

print("\n✅ All PDFs processed.")

📚 Found 0 PDF(s) in proc_final


📦 Processing PDFs: 0it [00:00, ?it/s]


✅ All PDFs processed.


2nd try ocr

In [ ]:
# 🛠 Required packages:
# pip install pymupdf pytesseract pillow tqdm

import fitz  # PyMuPDF
import pytesseract
from PIL import Image
from tqdm import tqdm
from pathlib import Path
import shutil
import io
import gc

# 📁 Define folders
input_folder = Path("failed_pdfs")
output_folder = Path("txt_final")
failed_folder = Path("failed_pdfs1")

output_folder.mkdir(parents=True, exist_ok=True)
failed_folder.mkdir(parents=True, exist_ok=True)

# 🧠 OCR with PyMuPDF
def ocr_with_fitz(pdf_path, txt_path):
    try:
        print(f"🔍 Processing {pdf_path.name}")
        doc = fitz.open(pdf_path)
        full_text = ""

        for page in tqdm(doc, desc=f"OCR: {pdf_path.name}", leave=False):
            pix = page.get_pixmap(dpi=300)
            img = Image.open(io.BytesIO(pix.tobytes()))
            text = pytesseract.image_to_string(img)
            full_text += text
            del img, pix, page
            gc.collect()

        with open(txt_path, "w", encoding="utf-8") as f:
            f.write(full_text)

        pdf_path.unlink()  # delete after success
        print(f"✅ Done and deleted: {pdf_path.name}")

    except Exception as e:
        print(f"❌ Failed to process {pdf_path.name}: {e}")
        try:
            shutil.move(str(pdf_path), failed_folder / pdf_path.name)
            print(f"📁 Moved to failed_pdfs: {pdf_path.name}")
        except Exception as move_error:
            print(f"⚠️ Could not move failed file: {move_error}")

# 🔁 Process all PDFs
pdf_files = list(input_folder.glob("*.pdf"))
print(f"📚 Found {len(pdf_files)} PDF(s) in {input_folder}")

for pdf_file in tqdm(pdf_files, desc="📦 Processing PDFs"):
    txt_file = output_folder / f"{pdf_file.stem}.txt"
    ocr_with_fitz(pdf_file, txt_file)

print("\n✅ All PDFs processed using PyMuPDF.")

📚 Found 742 PDF(s) in failed_pdfs


📦 Processing PDFs:   0%|          | 0/742 [00:00<?, ?it/s]

🔍 Processing 2015-Dec-31 - Telecom Italia SpA.pdf


📦 Processing PDFs:   0%|          | 1/742 [04:36<56:53:49, 276.42s/it]

✅ Done and deleted: 2015-Dec-31 - Telecom Italia SpA.pdf
🔍 Processing 2019-Dec-31 - Enagas SA.pdf


...

MuPDF error: format error: object is not a stream



MuPDF error: format error: object is not a stream



MuPDF error: format error: object is not a stream



MuPDF error: format error: object is not a stream



MuPDF error: library error: FT_New_Memory_Face(AAAABM+HelveticaNeue-Roman): unknown file format

